# NYC Parking Tickets: An Exploratory Analysis

In [1]:
## Note: To execute the code of this file the Kernel should be selected as PySpark

In [2]:
# Importing SparkSession Module
from pyspark.sql import SparkSession

# Creating SparkSession object
spark = SparkSession.builder.appName("NYC Parking Tickets").getOrCreate()

In [3]:
# Reading data
nyc_pt = spark.read.csv("/common_folder/nyc_parking/Parking_Violations_Issued_-_Fiscal_Year_2017.csv",
                        header=True, inferSchema = True)

### Inspecting Data

In [4]:
# Number of rows and column in dataset
print("Number of rows:",nyc_pt.count(),"\n","Number of Columns:",len(nyc_pt.columns))

Number of rows: 10803028 
 Number of Columns: 10


In [5]:
# Inspecting columns in dataset
nyc_pt.printSchema()

root
 |-- Summons Number: long (nullable = true)
 |-- Plate ID: string (nullable = true)
 |-- Registration State: string (nullable = true)
 |-- Issue Date: timestamp (nullable = true)
 |-- Violation Code: integer (nullable = true)
 |-- Vehicle Body Type: string (nullable = true)
 |-- Vehicle Make: string (nullable = true)
 |-- Violation Precinct: integer (nullable = true)
 |-- Issuer Precinct: integer (nullable = true)
 |-- Violation Time: string (nullable = true)



In [6]:
# Renaming columns for better handling of columns
nyc_pt = nyc_pt.withColumnRenamed("Summons Number", "Summons_Number")
nyc_pt = nyc_pt.withColumnRenamed("Plate ID", "Plate_ID")
nyc_pt = nyc_pt.withColumnRenamed("Registration State", "Registration_State")
nyc_pt = nyc_pt.withColumnRenamed("Issue Date", "Issue_Date")
nyc_pt = nyc_pt.withColumnRenamed("Violation Code", "Violation_Code")
nyc_pt = nyc_pt.withColumnRenamed("Vehicle Body Type", "Vehicle_Body_Type")
nyc_pt = nyc_pt.withColumnRenamed("Vehicle Make", "Vehicle_Make")
nyc_pt = nyc_pt.withColumnRenamed("Violation Precinct", "Violation_Precinct")
nyc_pt = nyc_pt.withColumnRenamed("Issuer Precinct", "Issuer_Precinct")
nyc_pt = nyc_pt.withColumnRenamed("Violation Time", "Violation_Time")

In [7]:
# Again inspecting after renaming columns to verify rename operation is done successfully
nyc_pt.show(5)

+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+
|Summons_Number|Plate_ID|Registration_State|         Issue_Date|Violation_Code|Vehicle_Body_Type|Vehicle_Make|Violation_Precinct|Issuer_Precinct|Violation_Time|
+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+
|    5092469481| GZH7067|                NY|2016-07-10 00:00:00|             7|             SUBN|       TOYOT|                 0|              0|         0143A|
|    5092451658| GZH7067|                NY|2016-07-08 00:00:00|             7|             SUBN|       TOYOT|                 0|              0|         0400P|
|    4006265037| FZX9232|                NY|2016-08-23 00:00:00|             5|             SUBN|        FORD|                 0|              0|         0233P|
|    8478629828| 66623ME|         

In [8]:
# Importing all functions from pyspark
import pyspark.sql.functions as func

In [9]:
# Counting missing value for each column
missing_per_col = []
for c in nyc_pt.columns:
    missing_per_col.append(nyc_pt.filter(func.isnull(nyc_pt[c])).count())
    
cols_missings = list(zip(nyc_pt.columns, missing_per_col))
cols_missings

[('Summons_Number', 0),
 ('Plate_ID', 0),
 ('Registration_State', 0),
 ('Issue_Date', 0),
 ('Violation_Code', 0),
 ('Vehicle_Body_Type', 0),
 ('Vehicle_Make', 0),
 ('Violation_Precinct', 0),
 ('Issuer_Precinct', 0),
 ('Violation_Time', 0)]

#### Inspecting columns

##### Issue_Date column

In [10]:
# Inspecting range of Issue date
nyc_pt.select(func.min("Issue_Date"), func.max("Issue_Date")).show()

+-------------------+-------------------+
|    min(Issue_Date)|    max(Issue_Date)|
+-------------------+-------------------+
|1972-03-30 00:00:00|2069-11-19 00:00:00|
+-------------------+-------------------+



In [11]:
# Different year of Issue_Date
nyc_pt.select(func.year("Issue_Date")).distinct().show(10)

+----------------+
|year(Issue_Date)|
+----------------+
|            1990|
|            2025|
|            1977|
|            2027|
|            2003|
|            2007|
|            2018|
|            1974|
|            2015|
|            2023|
+----------------+
only showing top 10 rows



## Examine the data

In [12]:
## Creating view of the dataframe to perform Sql operation on it
nyc_pt.createOrReplaceTempView("base_nyc_table")

# Inspecting View
spark.sql("select * from base_nyc_table").show(5)

+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+
|Summons_Number|Plate_ID|Registration_State|         Issue_Date|Violation_Code|Vehicle_Body_Type|Vehicle_Make|Violation_Precinct|Issuer_Precinct|Violation_Time|
+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+
|    5092469481| GZH7067|                NY|2016-07-10 00:00:00|             7|             SUBN|       TOYOT|                 0|              0|         0143A|
|    5092451658| GZH7067|                NY|2016-07-08 00:00:00|             7|             SUBN|       TOYOT|                 0|              0|         0400P|
|    4006265037| FZX9232|                NY|2016-08-23 00:00:00|             5|             SUBN|        FORD|                 0|              0|         0233P|
|    8478629828| 66623ME|         

#### <span style="color:blue">1. Find the total number of tickets for the year.</span>

#### Assumption: 
Dataset contains data from the year 2016 and 2017 i.e. financial year 2017.

For Analysis, we will consider whole data i.e. __data for year 2016 and 2017.__

In [13]:
# Counting total number of tickets for financial year 2017
ticket_freq_2017 = spark.sql("select count(*) as ticket_freq \
                              from base_nyc_table")
ticket_freq_2017.show()

+-----------+
|ticket_freq|
+-----------+
|   10803028|
+-----------+



#### <span style="color:blue">2. Find out the number of unique states from where the cars that got parking tickets came.</span>
__(Hint: Use the column 'Registration State'.)__

In [14]:
# Counting the number of unique states from where cars got parking tickets
unique_states_freq = spark.sql("select count(distinct Registration_State) as Unique_State_Count\
                                from base_nyc_table")
unique_states_freq.show()

+------------------+
|Unique_State_Count|
+------------------+
|                67|
+------------------+



In [15]:
# Inspecting unique states and identifying the number of parking tickets issued by each states
unique_states = spark.sql("select Registration_State, count(*) as Count\
                           from base_nyc_table\
                           group by Registration_State\
                           order by Count desc")
unique_states.show()

+------------------+-------+
|Registration_State|  Count|
+------------------+-------+
|                NY|8481061|
|                NJ| 925965|
|                PA| 285419|
|                FL| 144556|
|                CT| 141088|
|                MA|  85547|
|                IN|  80749|
|                VA|  72626|
|                MD|  61800|
|                NC|  55806|
|                IL|  37329|
|                GA|  36852|
|                99|  36625|
|                TX|  36516|
|                AZ|  26426|
|                OH|  25302|
|                CA|  24260|
|                SC|  21836|
|                ME|  21574|
|                MN|  18227|
+------------------+-------+
only showing top 20 rows



There is a __numeric entry '99'__ in the column, which should be corrected.<br>

Replacing it with the state having the maximum entries i.e. __replacing '99' with 'NY' which is having maximum entries.__

In [16]:
# Replacing '99' with 'NY'
nyc_pt_final = nyc_pt.withColumn("Registration_State", func.when(func.col("Registration_State")==99, 'NY')\
                           .otherwise(func.col("Registration_State")))

In [17]:
## Updating View
nyc_pt_final.createOrReplaceTempView("base_nyc_table")

In [18]:
# Again counting the number of unique states from where cars got parking tickets
corrected_unique_states_freq = spark.sql("select count(distinct Registration_State) as Unique_State_Count\
                                         from base_nyc_table")
corrected_unique_states_freq.show()

+------------------+
|Unique_State_Count|
+------------------+
|                66|
+------------------+



__The number of unique states from where the cars that got parking tickets are <span style="color:blue">66</span>__

## Aggregation tasks

#### <span style="color:blue">1. How often does each violation code occur? Display the frequency of the top five violation codes.</span>

In [19]:
# Finding frequency(count) of violation code
violation_code_freq = spark.sql("select Violation_Code, count(*) as Count \
                                 from base_nyc_table\
                                 group by Violation_Code\
                                 order by Count desc")

violation_code_freq.show(5) # Displaying top five violation codes

+--------------+-------+
|Violation_Code|  Count|
+--------------+-------+
|            21|1528588|
|            36|1400614|
|            38|1062304|
|            14| 893498|
|            20| 618593|
+--------------+-------+
only showing top 5 rows



#### <span style="color:blue">2. How often does each 'vehicle body type' get a parking ticket? How about the 'vehicle make'?</span>
__(Hint: Find the top 5 for both.)__

In [20]:
# Finding frequency(count) of parking ticket for vehicle body type
vehicle_body_type_freq = spark.sql("select Vehicle_Body_Type, count(*) as Count \
                                    from base_nyc_table\
                                    group by Vehicle_Body_Type\
                                    order by Count desc")

vehicle_body_type_freq.show(5) # Displaying top five frequency for vehicle body type

+-----------------+-------+
|Vehicle_Body_Type|  Count|
+-----------------+-------+
|             SUBN|3719802|
|             4DSD|3082020|
|              VAN|1411970|
|             DELV| 687330|
|              SDN| 438191|
+-----------------+-------+
only showing top 5 rows



In [21]:
# Finding frequency(count) of parking ticket for vehicle make
vehicle_make_freq = spark.sql("select Vehicle_Make, count(*) as Count \
                               from base_nyc_table\
                               group by Vehicle_Make\
                               order by Count desc")

vehicle_make_freq.show(5) # Displaying top five frequency for vehicle make

+------------+-------+
|Vehicle_Make|  Count|
+------------+-------+
|        FORD|1280958|
|       TOYOT|1211451|
|       HONDA|1079238|
|       NISSA| 918590|
|       CHEVR| 714655|
+------------+-------+
only showing top 5 rows



#### <span style="color:blue">3. A precinct is a police station that has a certain zone of the city under its command. Find the (5 highest) frequencies of tickets for each of the following:</span>
__(Hint: Print the top six entries after sorting.)__

##### <span style="color:red">3.1 'Violation Precinct' (This is the precinct of the zone where the violation occurred).</span>

In [22]:
# Finding frequency(count) of tickets for Violation Precinct
violation_precinct_freq = spark.sql("select Violation_Precinct, count(*) as Count \
                                     from base_nyc_table\
                                     group by Violation_Precinct\
                                     order by Count desc")

violation_precinct_freq.show(6) # Displaying top 6 frequencies for Violation Precinct

+------------------+-------+
|Violation_Precinct|  Count|
+------------------+-------+
|                 0|2072400|
|                19| 535671|
|                14| 352450|
|                 1| 331810|
|                18| 306920|
|               114| 296514|
+------------------+-------+
only showing top 6 rows



##### <span style="color:red">Using this, can you draw any insights for parking violations in any specific areas of the city?</span>

In [23]:
vio_pre_freq_area = spark.sql("select Registration_State, Violation_Precinct, count(*) as Count \
                               from base_nyc_table\
                               group by Registration_State, Violation_Precinct\
                               order by Count desc")
vio_pre_freq_area.show()

+------------------+------------------+-------+
|Registration_State|Violation_Precinct|  Count|
+------------------+------------------+-------+
|                NY|                 0|1749299|
|                NY|                19| 416114|
|                NY|               114| 249486|
|                NY|                 1| 236237|
|                NY|                14| 232881|
|                NY|                18| 200594|
|                NY|               109| 189639|
|                NY|                13| 172687|
|                NY|                70| 155195|
|                NY|               115| 143762|
|                NY|                61| 138470|
|                NY|                84| 134061|
|                NY|                17| 131612|
|                NY|               108| 130819|
|                NY|               112| 130674|
|                NY|                66| 129789|
|                NY|               103| 121269|
|                NY|                52| 

##### <span style="color:red">3.2 'Issuer Precinct' (This is the precinct that issued the ticket.)</span>

In [24]:
# Finding frequency(count) of tickets for Issuer Precinct
issuer_precinct_freq = spark.sql("select Issuer_Precinct, count(*) as Count \
                                  from base_nyc_table\
                                  group by Issuer_Precinct\
                                  order by Count desc")
issuer_precinct_freq.show(6) # Displaying top 6 frequencies for Issuer Precinct

+---------------+-------+
|Issuer_Precinct|  Count|
+---------------+-------+
|              0|2388479|
|             19| 521513|
|             14| 344977|
|              1| 321170|
|             18| 296553|
|            114| 289950|
+---------------+-------+
only showing top 6 rows



In [25]:
issuer_pre_freq_area = spark.sql("select Registration_State, Issuer_Precinct, count(*) as Count \
                                  from base_nyc_table\
                                  group by Registration_State, Issuer_Precinct\
                                  order by Count desc")

issuer_pre_freq_area.show()

+------------------+---------------+-------+
|Registration_State|Issuer_Precinct|  Count|
+------------------+---------------+-------+
|                NY|              0|1974434|
|                NY|             19| 405074|
|                NY|            114| 244046|
|                NY|              1| 228990|
|                NY|             14| 227876|
|                NY|             18| 192930|
|                NY|            109| 190153|
|                NY|             13| 168632|
|                NY|             70| 148760|
|                NJ|              0| 142846|
|                NY|            115| 139625|
|                NY|             61| 132406|
|                NY|             84| 129055|
|                NY|             17| 128809|
|                NY|            112| 126026|
|                NY|            108| 125469|
|                NY|             66| 119597|
|                NY|            103| 118662|
|                NY|             20| 118045|
|         

#### <span style="color:blue">4 Find the violation code frequencies for three precincts that have issued the most number of tickets. Do these precinct zones have an exceptionally high frequency of certain violation codes? Are these codes common across precincts?</span><br>
__(Hint: In the SQL view, use the 'where' attribute to filter among three precincts.)__

##### <span style="color:red">4.1 Find the violation code frequencies for three precincts that have issued the most number of tickets.</span>

In [26]:
# Finding top 3 precincts that have issued most number of tickets filtering erroneous entry of '0'
top3_precincts = spark.sql("select Issuer_Precinct,count(*) as Count\
                            from base_nyc_table\
                            where Issuer_Precinct != 0\
                            group by Issuer_Precinct\
                            order by Count desc")

top3_precincts.show(3)

+---------------+------+
|Issuer_Precinct| Count|
+---------------+------+
|             19|521513|
|             14|344977|
|              1|321170|
+---------------+------+
only showing top 3 rows



In [27]:
# Violation code frequency for highest Issuer precincts i.e. 19
top3_precincts_19 = spark.sql("select Issuer_Precinct,Violation_Code, count(*) as Count\
                               from base_nyc_table\
                               where Issuer_Precinct = 19\
                               group by Issuer_Precinct,Violation_Code\
                               order by Count desc")

top3_precincts_19.show(5)

+---------------+--------------+-----+
|Issuer_Precinct|Violation_Code|Count|
+---------------+--------------+-----+
|             19|            46|86390|
|             19|            37|72437|
|             19|            38|72344|
|             19|            14|57563|
|             19|            21|54700|
+---------------+--------------+-----+
only showing top 5 rows



In [28]:
# Violation code frequency for second highest Issuer precincts i.e. 14
top3_precincts_14 = spark.sql('select Issuer_Precinct,Violation_Code, count(*) as Count\
                               from base_nyc_table\
                               where Issuer_Precinct = 14\
                               group by Issuer_Precinct,Violation_Code\
                               order by Count desc')

top3_precincts_14.show(5)

+---------------+--------------+-----+
|Issuer_Precinct|Violation_Code|Count|
+---------------+--------------+-----+
|             14|            14|73837|
|             14|            69|58026|
|             14|            31|39857|
|             14|            47|30540|
|             14|            42|20663|
+---------------+--------------+-----+
only showing top 5 rows



In [29]:
# Violation code frequency for third highest Issuer precincts i.e. 1
top3_precincts_1 = spark.sql('select Issuer_Precinct,Violation_Code, count(*) as Count\
                               from base_nyc_table\
                               where Issuer_Precinct = 1\
                               group by Issuer_Precinct,Violation_Code\
                               order by Count desc')

top3_precincts_1.show(5)

+---------------+--------------+-----+
|Issuer_Precinct|Violation_Code|Count|
+---------------+--------------+-----+
|              1|            14|73522|
|              1|            16|38937|
|              1|            20|27841|
|              1|            46|22534|
|              1|            38|16989|
+---------------+--------------+-----+
only showing top 5 rows



##### <span style="color:red">4.2 Do these precinct zones have an exceptionally high frequency of certain violation codes?</span>

- For Issuer_Precinct - 19
    - Top five violation code are
            46|86390|
            37|72437|
            38|72344|
            14|57563|
            21|54700|
    - We can see for violation code 46, frequency is on higher side as compared to other violation code.

- For Issuer_Precinct - 14
    - Top five violation code are
            14|73837|
            69|58026|
            31|39857|
            47|30540|
            42|20663|
    - We can see for violation code 14, frequency is on higher side as compared to other violation code.

- For Issuer_Precinct - 1
    - Top five violation code are
            14|73522|
            16|38937|
            20|27841|
            46|22534|
            38|16989|
    - We can see for violation code 1, frequency is exceptionally high as compared to other violation code.

##### <span style="color:red">4.3 Are these codes common across precincts?</span>

#### <span style="color:blue">Assumption</span>
To answer this we are considering top 5 violation code for top three precincts

#### <span style="color:blue">Conclusion</span>
From the above summary, Considering top 5 violation code for top three precincts that have issued the most number of tickets, 

we can observe that __violation code 14 is common across top three precincts__

#### <span style="color:blue">5. Find out the properties of parking violations across different times of the day:</span>

##### <span style="color:red">5.1 Find a way to deal with missing values, if any.</span>

In [33]:
# Importing specific functions to count nan
from pyspark.sql.functions import isnan, when, count, col

# Inspecting Null and NaN values in Violation_Time
nyc_pt_final.select([count(when(isnan("Violation_Time") | col("Violation_Time").isNull(), "Violation_Time"))]).show()

+--------------------------------------------------------------------------------------------+
|count(CASE WHEN (isnan(Violation_Time) OR (Violation_Time IS NULL)) THEN Violation_Time END)|
+--------------------------------------------------------------------------------------------+
|                                                                                           0|
+--------------------------------------------------------------------------------------------+



In [34]:
print("Count nan in Violation_Time",nyc_pt_final.filter(nyc_pt_final.Violation_Time == 'nan').count())

Count nan in Violation_Time 63


Missing value can be of two types __nan and Null__ values
- When we inspected for Null values in the data, there are __no Null values present.__
- When we inspected for NaN values in the data, there are __no NaN values present.__
- But there are columns which contain values as __'nan' for some rows__. If we treat them as null then __we can filter out these values from dataframe.__

##### <span style="color:red">5.2 The Violation Time field is specified in a strange format. Find a way to make this a time attribute that you can use to divide into groups.</span>

In [35]:
# Extracting Hour, Minute and "AM/PM" from Violation_time column to making time attribute
df_time = nyc_pt_final.withColumn("LeftTime", func.substring(func.col("Violation_Time"),1, 2).cast("int"))\
                      .withColumn("RightTime", func.substring(func.col("Violation_Time"),3, 2).cast("int"))\
                      .withColumn("AMPM", func.substring(func.col("Violation_Time"),5, 1))

In [36]:
# Inspecting the extracted Hour, Minute and 'AM/PM' column to verify that extraction was successful
df_time.select("Violation_Time","LeftTime","RightTime","AMPM").show(5)

+--------------+--------+---------+----+
|Violation_Time|LeftTime|RightTime|AMPM|
+--------------+--------+---------+----+
|         0143A|       1|       43|   A|
|         0400P|       4|        0|   P|
|         0233P|       2|       33|   P|
|         1120A|      11|       20|   A|
|         0555P|       5|       55|   P|
+--------------+--------+---------+----+
only showing top 5 rows



In [38]:
# Converting Hour from 12 Hour format to 24 Hour format
df_time2 = df_time.withColumn("Left24", func.when((df_time.AMPM=="P") & (df_time.LeftTime < 12), df_time.LeftTime+12)\
                                            .when((df_time.AMPM=="A") & (df_time.LeftTime >= 12), df_time.LeftTime-12)\
                                            .otherwise(df_time.LeftTime))

df_time2.select("Left24").show(5)

+------+
|Left24|
+------+
|     1|
|    16|
|    14|
|    11|
|    17|
+------+
only showing top 5 rows



In [39]:
# Concatenating Hour and Minute column to make complete time attribute in 24 hour format
df_time3 = df_time2.withColumn("VT24", func.concat_ws(":",df_time2.Left24, df_time2.RightTime))

# Inspecting time in string vs time in 24 hour format
df_time3.select("Violation_Time","VT24").show(5)

+--------------+-----+
|Violation_Time| VT24|
+--------------+-----+
|         0143A| 1:43|
|         0400P| 16:0|
|         0233P|14:33|
|         1120A|11:20|
|         0555P|17:55|
+--------------+-----+
only showing top 5 rows



##### <span style="color:red">5.3 Divide 24 hours into six equal discrete bins of time. Choose the intervals as you see fit. For each of these groups, find the three most commonly occurring violations.</span>

#### <span style="color:blue">Assumption</span>
Assuming intervals as follows:
  - __0AM - 4AM as Midnight__
  - __4AM - 8AM as Early_Morning__
  - __8AM - 12PM as Morning__
  - __12PM - 16PM (4PM) as After_Noon__
  - __16PM (4PM) - 20PM (8PM) as Evening__
  - __20PM (8PM) - 24AM(12AM) as Night__

In [43]:
# Creating view to accomodate changes made in dataframe by adding time attribute 
df_time3.createOrReplaceTempView("nyc_with_time")

# Dividing the 24 hour into 6 equal discrete time bin
df_time4 = spark.sql('select Violation_Code,VT24,\
                            case \
                              when (Left24>=0 and Left24<4) then "Midnight" \
                              when (Left24>=4 and Left24<8) then "Early_Morning" \
                              when (Left24>=8 and Left24<12) then "Morning" \
                              when (Left24>=12 and Left24<16) then "After_Noon" \
                              when (Left24>=16 and Left24<20) then "Evening" \
                              else "Night" \
                            end as VT_BIN \
                      from nyc_with_time')

df_time4.show(5)

+--------------+-----+----------+
|Violation_Code| VT24|    VT_BIN|
+--------------+-----+----------+
|             7| 1:43|  Midnight|
|             7| 16:0|   Evening|
|             5|14:33|After_Noon|
|            47|11:20|   Morning|
|            69|17:55|   Evening|
+--------------+-----+----------+
only showing top 5 rows



In [44]:
# Creating view again to accomodate changes made in dataframe by adding discrete time bin
df_time4.createOrReplaceTempView("nyc_with_intervals")

In [45]:
# Finding three most commonly occurring violations for Midnight (0AM - 4AM) interval
nyc_midnight = spark.sql('select Violation_Code,count(*) as Count\
                          from nyc_with_intervals\
                          where VT_BIN = "Midnight" \
                          group by Violation_Code order by Count desc')
nyc_midnight.show(3)

+--------------+-----+
|Violation_Code|Count|
+--------------+-----+
|            21|77461|
|            40|50948|
|            78|32243|
+--------------+-----+
only showing top 3 rows



In [46]:
# Finding three most commonly occurring violations for Early_Morning (4AM - 8AM) interval
nyc_early_morning = spark.sql('select Violation_Code,count(*) as Count\
                               from nyc_with_intervals\
                               where VT_BIN = "Early_Morning" \
                               group by Violation_Code order by Count desc')

nyc_early_morning.show(3)

+--------------+------+
|Violation_Code| Count|
+--------------+------+
|            14|141276|
|            21|119469|
|            40|112187|
+--------------+------+
only showing top 3 rows



In [47]:
# Finding three most commonly occurring violations for Morning (8AM - 12PM) interval
nyc_morning = spark.sql('select Violation_Code,count(*) as Count\
                         from nyc_with_intervals\
                         where VT_BIN = "Morning" \
                         group by Violation_Code order by Count desc')
nyc_morning.show(3)

+--------------+-------+
|Violation_Code|  Count|
+--------------+-------+
|            21|1182691|
|            36| 751422|
|            38| 346518|
+--------------+-------+
only showing top 3 rows



In [48]:
# Finding three most commonly occurring violations for After_Noon (12PM - 4PM) interval
nyc_after_noon = spark.sql('select Violation_Code,count(*) as Count\
                            from nyc_with_intervals\
                            where VT_BIN = "After_Noon" \
                            group by Violation_Code order by Count desc')

nyc_after_noon.show(3)

+--------------+------+
|Violation_Code| Count|
+--------------+------+
|            36|588395|
|            38|462758|
|            37|337075|
+--------------+------+
only showing top 3 rows



In [49]:
# Finding three most commonly occurring violations for Evening (4PM - 8PM) interval
nyc_evening = spark.sql('select Violation_Code,count(*) as Count\
                         from nyc_with_intervals\
                         where VT_BIN = "Evening" \
                         group by Violation_Code order by Count desc')

nyc_evening.show(3)

+--------------+------+
|Violation_Code| Count|
+--------------+------+
|            38|203232|
|            37|145784|
|            14|144749|
+--------------+------+
only showing top 3 rows



In [50]:
# Finding three most commonly occurring violations for Night (8PM - 12AM) interval
nyc_night = spark.sql('select Violation_Code,count(*) as Count\
                       from nyc_with_intervals\
                       where VT_BIN = "Night" \
                       group by Violation_Code order by Count desc')

nyc_night.show(3)

+--------------+-----+
|Violation_Code|Count|
+--------------+-----+
|             7|65593|
|            38|47032|
|            14|44787|
+--------------+-----+
only showing top 3 rows



##### <span style="color:red">5.4 Now, try another direction. For the three most commonly occurring violation codes, find the most common time of the day (in terms of the bins from the previous part).</span>

In [52]:
# Finding three most commonly occuring Violation Code
nyc_top3_vc = spark.sql('select Violation_Code,count(*) as Count\
                         from nyc_with_intervals\
                         group by Violation_Code order by Count desc')

nyc_top3_vc.show(3)

+--------------+-------+
|Violation_Code|  Count|
+--------------+-------+
|            21|1528588|
|            36|1400614|
|            38|1062304|
+--------------+-------+
only showing top 3 rows



The most __commonly occuring Violation Codes are 21, 36 and 38__. Lets find the most common time of the day for them

In [53]:
# Identifying most common time of the day when violation code is 21
nyc_vc_21 = spark.sql('select VT_BIN, count(*) as Count\
                       from nyc_with_intervals\
                       where Violation_Code = "21" \
                       group by VT_BIN order by Count desc')

nyc_vc_21.show(3)

+-------------+-------+
|       VT_BIN|  Count|
+-------------+-------+
|      Morning|1182691|
|   After_Noon| 148014|
|Early_Morning| 119469|
+-------------+-------+
only showing top 3 rows



In [54]:
# Identifying most common time of the day when violation code is 36
nyc_vc_36 = spark.sql('select VT_BIN, count(*) as Count\
                       from nyc_with_intervals\
                       where Violation_Code = "36" \
                       group by VT_BIN order by Count desc')
nyc_vc_36.show(3)

+-------------+------+
|       VT_BIN| Count|
+-------------+------+
|      Morning|751422|
|   After_Noon|588395|
|Early_Morning| 33939|
+-------------+------+
only showing top 3 rows



In [55]:
# Identifying most common time of the day when violation code is 38
nyc_vc_38 = spark.sql('select VT_BIN, count(*) as Count\
                       from nyc_with_intervals\
                       where Violation_Code = "38" \
                       group by VT_BIN order by Count desc')
nyc_vc_38.show(3)

+----------+------+
|    VT_BIN| Count|
+----------+------+
|After_Noon|462758|
|   Morning|346518|
|   Evening|203232|
+----------+------+
only showing top 3 rows



#### Observation

- For Violation_Code - 21, most common time of the day is __Morning (8AM to 12PM)__
- For Violation_Code - 36, most common time of the day is __Morning (8AM to 12PM)__
- For Violation_Code - 38, most common time of the day is __After_Noon (12PM to 4PM)__

#### <span style="color:blue">6. Let’s try and find some seasonality in this data:</span>

##### <span style="color:red">6.1 First, divide the year into a certain number of seasons, and find the frequencies of tickets for each season.</span>
__(Hint: Use Issue Date to segregate into seasons.)__

In [56]:
# Extracting month from the year to divide into seasons
mon_df = nyc_pt_final.withColumn("Issue_Month", func.month(nyc_pt_final.Issue_Date))
mon_df.show(5)

+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+-----------+
|Summons_Number|Plate_ID|Registration_State|         Issue_Date|Violation_Code|Vehicle_Body_Type|Vehicle_Make|Violation_Precinct|Issuer_Precinct|Violation_Time|Issue_Month|
+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+-----------+
|    5092469481| GZH7067|                NY|2016-07-10 00:00:00|             7|             SUBN|       TOYOT|                 0|              0|         0143A|          7|
|    5092451658| GZH7067|                NY|2016-07-08 00:00:00|             7|             SUBN|       TOYOT|                 0|              0|         0400P|          7|
|    4006265037| FZX9232|                NY|2016-08-23 00:00:00|             5|             SUBN|        FORD|                 0|      

#### <span style="color:blue">Assumption</span>
The Seasons have been assumed as :
* __Fall Season : September(9), October(10), November(11)__
* __Winter Season : December(12), January(1), February(2)__
* __Spring Season : March(3), April(4), May(5)__
* __Summer Season : June(6), July(7), August(8)__

In [59]:
# Creating view to accomodate changes made in dataframe by extracting month from the Issue_Date
mon_df.createOrReplaceTempView("nyc_with_month")

# Segregating year into 4 seasons
df_season = spark.sql('select Violation_Code,Issue_Month,\
                              case \
                                  when (Issue_Month=9 or Issue_Month=10 or Issue_Month=11) then "Fall" \
                                  when (Issue_Month=12 or Issue_Month=1 or Issue_Month=2) then "Winter" \
                                  when (Issue_Month=3 or Issue_Month=4 or Issue_Month=5) then "Spring" \
                                  else "Summer" \
                              end as Season \
                       from nyc_with_month')

df_season.show(10)

+--------------+-----------+------+
|Violation_Code|Issue_Month|Season|
+--------------+-----------+------+
|             7|          7|Summer|
|             7|          7|Summer|
|             5|          8|Summer|
|            47|          6|Summer|
|            69|         11|  Fall|
|             7|          6|Summer|
|            40|          8|Summer|
|            36|         12|Winter|
|            36|         11|  Fall|
|             5|         10|  Fall|
+--------------+-----------+------+
only showing top 10 rows



##### To Find the frequency of tickets for each season

In [61]:
# Creating view to accomodate changes made in dataframe by segregating year into seasons
df_season.createOrReplaceTempView("nyc_with_season")

# Finding frequency of tickets for each season
nyc_ticket_season_freq = spark.sql('select Season, count(*) as Count\
                                    from nyc_with_season\
                                    group by Season order by Count desc')

nyc_ticket_season_freq.show()

+------+-------+
|Season|  Count|
+------+-------+
|Spring|2880687|
|  Fall|2830802|
|Summer|2606208|
|Winter|2485331|
+------+-------+



##### <span style="color:red">6.2 then, find the three most common violations for each of these seasons.</span>
__(Hint: You can use an approach similar to the one mentioned in the hint for question 4.)__

In [62]:
# Finding three most common violations for Winter(Month 12,1,2) season
nyc_winter = spark.sql('select Violation_Code, count(*) as Count\
                        from nyc_with_season\
                        where Season="Winter"\
                        group by Violation_Code order by Count desc')

nyc_winter.show(3)

+--------------+------+
|Violation_Code| Count|
+--------------+------+
|            21|362341|
|            36|359338|
|            38|259723|
+--------------+------+
only showing top 3 rows



In [63]:
# Finding three most common violations for Spring(Month 3,4,5) season
nyc_spring = spark.sql('select Violation_Code, count(*) as Count\
                        from nyc_with_season\
                        where Season="Spring"\
                        group by Violation_Code order by Count desc')

nyc_spring.show(3)

+--------------+------+
|Violation_Code| Count|
+--------------+------+
|            21|402807|
|            36|344834|
|            38|271192|
+--------------+------+
only showing top 3 rows



In [64]:
# Finding three most common violations for Summer(Month 6,7,8) season
nyc_summer = spark.sql('select Violation_Code, count(*) as Count\
                        from nyc_with_season\
                        where Season="Summer"\
                        group by Violation_Code order by Count desc')

nyc_summer.show(3)

+--------------+------+
|Violation_Code| Count|
+--------------+------+
|            21|405961|
|            38|247561|
|            36|240396|
+--------------+------+
only showing top 3 rows



In [65]:
# Finding three most common violations for Fall(Month 9,10,11) season
nyc_fall = spark.sql('select Violation_Code, count(*) as Count\
                      from nyc_with_season\
                      where Season="Fall"\
                      group by Violation_Code order by Count desc')

nyc_fall.show(3)

+--------------+------+
|Violation_Code| Count|
+--------------+------+
|            36|456046|
|            21|357479|
|            38|283828|
+--------------+------+
only showing top 3 rows



#### Observation

- __For Winter(Month 12,1,2) season three most common violations with there frequency are are:__
            | Violation Code | No. of Tickets | 
            |     21         |    362341      | 
            |     36         |    359338      |
            |     38         |    259723      |    

- __For Spring(Month 3,4,5) season three most common violations are:__
            | Violation Code | No. of Tickets | 
            |     21         |    402807      |
            |     36         |    344834      |
            |     38         |    271192      |

- __For Summer(Month 6,7,8) season three most common violations are:__
            | Violation Code | No. of Tickets | 
            |     21         |    405961      |
            |     38         |    247561      |
            |     36         |    240396      |

- __For Fall(Month 9,10,11) season three most common violations are:__
            | Violation Code | No. of Tickets | 
            |     36         |    456046      |
            |     21         |    357479      |
            |     38         |    283828      |

#### <span style="color:blue">7. The fines collected from all the instances of parking violation constitute a source of revenue for the NYC Police Department. Let’s take an example of estimating this for the three most commonly occurring codes:</span>

##### <span style="color:red">7.1 Find the total occurrences of the three most common violation codes.</span>

In [67]:
# Identifying total occurrences of the three most common violation codes
Count_df = spark.sql('select Violation_Code, count(*) as Count\
                      from nyc_with_season\
                      group by Violation_Code order by Count desc')

Count_df.show(3)

+--------------+-------+
|Violation_Code|  Count|
+--------------+-------+
|            21|1528588|
|            36|1400614|
|            38|1062304|
+--------------+-------+
only showing top 3 rows



__Three most commonly occuring violation codes are 21,36 and 38__

##### <span style="color:red">7.2 then, visit the website:</span>
http://www1.nyc.gov/site/finance/vehicles/services-violation-codes.page
It lists the fines associated with different violation codes. They’re divided into two categories: one for the highest-density locations in the city and the other for the rest of the city. __For the sake of simplicity, take the average of the two.__

#### Fine calculation
- __For Violation Code 21__
    - Fines associated are highest-density locations in the city - `$65` & other for the rest of the city -`$45`
    - Taking average (65+45)/2 we get __fine for violation code 21 is $55__


- __For Violation Code 36__
    - Fines associated are highest-density locations in the city - `$50` & other for the rest of the city -`$50`
    - Taking average (50+50)/2 we get __fine for violation code 36 is $50__
    
    
- __For Violation Code 38__
    - Fines associated are highest-density locations in the city - `$65` & other for the rest of the city -`$35`
    - Taking average (65+35)/2 we get __fine for violation code 38 is $50__

##### <span style="color:red">7.3 Using this information, find the total amount collected for the three violation codes with the maximum tickets. State the code that has the highest total collection.</span>

In [68]:
# Creating view
Count_df.createOrReplaceTempView("countTable")

In [69]:
# Total Amount collected for Violation Code 21
nyc_fine_vc21 = spark.sql('select Count, Count*55 as TotalFine\
                           from countTable\
                           where Violation_Code="21"')

nyc_fine_vc21.show()

+-------+---------+
|  Count|TotalFine|
+-------+---------+
|1528588| 84072340|
+-------+---------+



In [70]:
# Total Amount collected for Violation Code 36
nyc_fine_vc36 = spark.sql('select Count, Count*50 as TotalFine\
                           from countTable\
                           where Violation_Code="36"')

nyc_fine_vc36.show()

+-------+---------+
|  Count|TotalFine|
+-------+---------+
|1400614| 70030700|
+-------+---------+



In [71]:
# Total Amount collected for Violation Code 38
nyc_fine_vc38 = spark.sql('select Count, Count*50 as TotalFine\
                           from countTable\
                           where Violation_Code ="38"')

nyc_fine_vc38.show()

+-------+---------+
|  Count|TotalFine|
+-------+---------+
|1062304| 53115200|
+-------+---------+



##### <span style="color:red">7.4 What can you intuitively infer from these findings?</span>

#### Observation and Findings
Summary for most commonly occuring violation codes i.e. 21,36 and 38

| Violation Code | No. of Tickets | Total Fine |
| --- | --- | --- |
| 21 | 1528588 | 84072340 |
| 36 | 1400614 | 70030700 |
| 38 | 1062304 | 53115200 |

- __Violation Code 21 has the highest Total Collection__

In [72]:
# Stopping SparkSession object
spark.stop()